In [10]:
from pathlib import Path
import re
import pandas as pd
import numpy as np
import json
import pickle

# directory containing game parquet files (relative to this notebook)
path = Path('..') / 'data' / 'processed' / 'game'
files = sorted(path.glob('*.parquet'))
# load exactly one DataFrame per season (keep the first file encountered for each year)
games = {}  # maps YYYY -> DataFrame
for f in files:
    # read parquet first so any I/O/parquet-engine errors surface immediately
    df = pd.read_parquet(f)

    # --- reorder columns: week, team names, team points, home_*, away_*, rest ---
    week_col = None
    for candidate in ('week', 'week_num', 'week_number'):
        if candidate in df.columns:
            week_col = candidate
            break
    team_cols = [c for c in ('home_team', 'away_team') if c in df.columns]
    points_cols = [c for c in ('home_points', 'away_points') if c in df.columns]
    home_pref = [c for c in df.columns if c.startswith('home_') and c not in points_cols and c not in team_cols]
    away_pref = [c for c in df.columns if c.startswith('away_') and c not in points_cols and c not in team_cols]
    used = set()
    if week_col:
        used.add(week_col)
    used.update(team_cols)
    used.update(points_cols)
    used.update(home_pref)
    used.update(away_pref)
    remaining = [c for c in df.columns if c not in used]
    new_order = []
    if week_col:
        new_order.append(week_col)
    new_order.extend(team_cols)
    new_order.extend(points_cols)
    new_order.extend(home_pref)
    new_order.extend(away_pref)
    new_order.extend(remaining)
    new_order = [c for c in new_order if c in df.columns]
    df = df.loc[:, new_order]
    # --- end reorder ---

    fname = f.name
    m = re.search(r'(19|20)\d{2}', fname)
    if m:
        year = m.group(0)
    else:
        # fallback: use filename without suffix
        year = fname.rsplit('.parquet', 1)[0]
    # normalize to a 4-digit year string when possible
    year_str = year if (isinstance(year, str) and len(str(year)) == 4 and str(year).isdigit()) else str(year)
    # if we've already loaded a DataFrame for this season, skip further files
    if year_str in games:
        # skip duplicates for the same season
        continue
    # register the DataFrame for this season
    games[year_str] = df
    # expose short-name globals: games_YY (e.g. games_16) and games_YYYY
    short = year_str[-2:] if year_str.isdigit() and len(year_str) == 4 else year_str
    globals()[f'games_{year_str}'] = df
    globals()[f'games_{short}'] = df
    # backward compatibility: also expose df_YYYY and df_YY aliases
    globals()[f'df_{year_str}'] = df
    globals()[f'df_{short}'] = df
    print(f'Loaded {f} -> games_{year_str} (alias games_{short}, df_{year_str}), shape={df.shape}')
# convenience alias: dfs points to the per-season mapping we just built
dfs = games
# use dfs['2017'] or games_17 / df_17 as needed

Loaded ../data/processed/game/games_wide_2016_regular.parquet -> games_2016 (alias games_16, df_2016), shape=(832, 196)
Loaded ../data/processed/game/games_wide_2017_regular.parquet -> games_2017 (alias games_17, df_2017), shape=(834, 196)
Loaded ../data/processed/game/games_wide_2018_regular.parquet -> games_2018 (alias games_18, df_2018), shape=(845, 196)
Loaded ../data/processed/game/games_wide_2019_regular.parquet -> games_2019 (alias games_19, df_2019), shape=(848, 196)
Loaded ../data/processed/game/games_wide_2020_regular.parquet -> games_2020 (alias games_20, df_2020), shape=(542, 196)
Loaded ../data/processed/game/games_wide_2021_regular.parquet -> games_2021 (alias games_21, df_2021), shape=(849, 196)
Loaded ../data/processed/game/games_wide_2022_regular.parquet -> games_2022 (alias games_22, df_2022), shape=(854, 196)
Loaded ../data/processed/game/games_wide_2023_regular.parquet -> games_2023 (alias games_23, df_2023), shape=(868, 196)
Loaded ../data/processed/game/games_wide

In [3]:
drop_cols = ['home_team_x','away_team_x','home_team_y','away_team_y','away_interceptionTDs','home_interceptonTDs','home_puntReturnYards','away_puntReturnYards','away_kickReturnYards','home_kickReturnYards','home_kickReturnTDs','away_kickReturnTDs','favorite','season', 'season_type','start_date','home_conference','venue','neutral_site','conference_game','away_conference','home_KickReturns','away_kickReturns','home_puntReturns','home_puntReturnTDs','away_puntReturnTDs','home_interceptionYards','away_interceptionYards','home_interceptionTDs','away_interceptonTDs','away_puntReturns','home_puntReturnTDs','away_puntReturnTDs','home_team_conference','away_team_conference']
dfs = [games_2016, games_2017, games_2018, games_2019, games_2020, games_2021, games_2022, games_2023, games_2024]
for df in dfs:
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    print(df.shape)

(832, 166)
(834, 166)
(845, 166)
(848, 166)
(542, 166)
(849, 166)
(854, 166)
(868, 166)
(874, 166)


In [40]:
# Count nulls per column and overall for a target DataFrame
def count_nulls(df):
    """Return (summary_df, total_nulls). summary_df has columns: null_count, null_pct."""
    nulls = df.isna().sum()
    total_nulls = int(nulls.sum())
    pct = (nulls / len(df)) * 100 if len(df) > 0 else 0
    summary = pd.DataFrame({'null_count': nulls.astype(int), 'null_pct': pct})
    summary = summary.sort_values('null_count', ascending=False)
    return summary, total_nulls

# Auto-detect a sensible target dataframe (adjust candidates if you use other names)
for candidate in ('df_a', 'df', 'df_16', 'df_2016', 'df_2024', 'df_24'):
    if candidate in globals():
        target_name = candidate
        break
else:
    raise NameError('No target dataframe found. Define `df` or `df_a` or df_YYYY beforehand.')

target_df = globals()[target_name]
summary, total_nulls = count_nulls(target_df)
print(f'Target: {target_name} | rows={len(target_df)} cols={target_df.shape[1]}')
print(f'Total null values: {total_nulls}')
print('Top 20 columns by null count:')
print(summary.head(20).to_string())

# Optionally save summary to CSV/JSON for later inspection (uncomment to enable)
# out_csv = Path('..') / 'data' / 'processed' / f'nulls_{target_name}.csv'
# summary.to_csv(out_csv)
# print('Saved null-summary ->', out_csv)

Target: df | rows=874 cols=166
Total null values: 1950
Top 20 columns by null count:
                        null_count   null_pct
away_passesIntercepted         434  49.656751
home_passesIntercepted         358  40.961098
away_totalFumbles              184  21.052632
home_kickReturns               178  20.366133
home_totalFumbles              172  19.679634
away_kickingPoints              22   2.517162
total                           17   1.945080
spread                          16   1.830664
home_tacklesForLoss              7   0.800915
home_qbHurries                   7   0.800915
home_sacks                       7   0.800915
home_tackles                     7   0.800915
home_passesDeflected             7   0.800915
home_defensiveTDs                7   0.800915
home_kickingPoints               6   0.686499
away_passesDeflected             6   0.686499
away_tacklesForLoss              6   0.686499
away_tackles                     6   0.686499
away_sacks                       6   0.68

In [12]:
# Fill all nulls (NA/NaN) with 0 for detected DataFrames
import re
def fill_all_nulls():
    """Detect DataFrame objects in common locations (dfs dict/list, df_YYYY names)
    and replace all missing values with 0 in-place. Returns a list of report tuples.
    """
    candidates = []
    # check `dfs` if present (can be dict or list)
    if 'dfs' in globals():
        obj = globals()['dfs']
        if isinstance(obj, dict):
            for k,v in obj.items():
                if isinstance(v, pd.DataFrame):
                    candidates.append((k,v))
        elif isinstance(obj, list):
            for i,v in enumerate(obj):
                if isinstance(v, pd.DataFrame):
                    candidates.append((f'dfs[{i}]', v))
    # discover globals named df_YY or df_YYYY
    for name, val in list(globals().items()):
        if re.match(r'^df_\d{2,4}$', name) and isinstance(val, pd.DataFrame):
            # avoid duplicate entries
            if not any(name == n for n,_ in candidates):
                candidates.append((name, val))
    if not candidates:
        raise NameError('No DataFrame candidates found to fill nulls. Define `df`, `df_a`, or `dfs` first.')
    reports = []
    for name, df in candidates:
        before = int(df.isna().sum().sum())
        # perform in-place fill; this will replace NA in all dtypes with 0
        df.fillna(0, inplace=True)
        after = int(df.isna().sum().sum())
        reports.append((name, before, after, df.shape))
    return reports

# Run the operation and print a compact report
reports = fill_all_nulls()
for name, before, after, shape in reports:
    print(f'{name}: nulls before={before}, after={after}, shape={shape}')

# signal completion to the notebook session
globals()['nulls_filled'] = True
print('Completed filling nulls; set nulls_filled=True')

2016: nulls before=0, after=0, shape=(832, 196)
2017: nulls before=0, after=0, shape=(834, 196)
2018: nulls before=0, after=0, shape=(845, 196)
2019: nulls before=0, after=0, shape=(848, 196)
2020: nulls before=0, after=0, shape=(542, 196)
2021: nulls before=0, after=0, shape=(849, 196)
2022: nulls before=0, after=0, shape=(854, 196)
2023: nulls before=0, after=0, shape=(868, 196)
2024: nulls before=0, after=0, shape=(874, 196)
df_2016: nulls before=0, after=0, shape=(832, 196)
df_16: nulls before=0, after=0, shape=(832, 196)
df_2017: nulls before=0, after=0, shape=(834, 196)
df_17: nulls before=0, after=0, shape=(834, 196)
df_2018: nulls before=0, after=0, shape=(845, 196)
df_18: nulls before=0, after=0, shape=(845, 196)
df_2019: nulls before=0, after=0, shape=(848, 196)
df_19: nulls before=0, after=0, shape=(848, 196)
df_2020: nulls before=0, after=0, shape=(542, 196)
df_20: nulls before=0, after=0, shape=(542, 196)
df_2021: nulls before=0, after=0, shape=(849, 196)
df_21: nulls befo

In [13]:
# Save ONE DataFrame per season as games_YYYY.parquet into ../data/processed/structured
from pathlib import Path
import re
out_dir = Path('..') / 'data' / 'processed' / 'structured'
out_dir.mkdir(parents=True, exist_ok=True)
saved = []

def _sanitize_df_for_parquet(df):
    """Return a copy of df with column names as str and decode/normalize problematic values.
    Specifically coerce `home_team` / `away_team` to str and ensure object columns are
    decoded from bytes and cast to string to avoid pyarrow binary/inference issues.
    """
    d = df.copy()
    # ensure column names are strings
    d.columns = [str(c) for c in d.columns]

    # Coerce known problematic team columns to string (handles ints, bytes, NaN)
    for col in ('home_team', 'away_team'):
        if col in d.columns:
            d[col] = d[col].where(d[col].notnull(), '').astype(str)

    # decode bytes/bytearray in object columns and cast to str (preserve empties)
    obj_cols = d.select_dtypes(include=['object']).columns.tolist()
    if obj_cols:
        for c in obj_cols:
            def _maybe_decode(x):
                if isinstance(x, (bytes, bytearray)):
                    try:
                        return x.decode('utf-8')
                    except Exception:
                        return str(x)
                return x
            d[c] = d[c].apply(_maybe_decode)
            # replace NaN with empty string then cast to str to avoid mixed-type columns
            d[c] = d[c].where(d[c].notnull(), '').astype(str)
    return d

# If `dfs` is a dict mapping year->df, prefer that (common earlier in this notebook)
if 'dfs' in globals():
    container = globals()['dfs']
    if isinstance(container, dict):
        for key, df in container.items():
            if not isinstance(df, pd.DataFrame):
                continue
            k = str(key)
            m = re.search(r'(19|20)\d{2}', k)
            year = m.group(0) if m else re.sub(r'[^0-9]', '', k) or k
            out_path = out_dir / f'games_{year}.parquet'
            try:
                d = _sanitize_df_for_parquet(df)
                # write parquet (pyarrow/fastparquet); index not required
                d.to_parquet(out_path.as_posix(), index=False)
                saved.append(str(out_path))
            except Exception as e:
                print(f'Parquet write failed for {year}: {e!s}')
    elif isinstance(container, list):
        # if dfs is a list, attempt to save using games_YYYY globals instead (fallback below)
        pass
# Fallback: look for globals named games_YYYY or games_YY and save each once
seen = set()
for name, val in list(globals().items()):
    if re.match(r'^(games|df)_(19|20)\d{2}$', name) and isinstance(val, pd.DataFrame):
        year = name.split('_',1)[1]
        if year in seen:
            continue
        out_path = out_dir / f'games_{year}.parquet'
        try:
            d = _sanitize_df_for_parquet(val)
            d.to_parquet(out_path.as_posix(), index=False)
            saved.append(str(out_path))
            seen.add(year)
        except Exception as e:
            print(f'Parquet write failed for {year}: {e!s}')
# Deduplicate saved list and report
saved = sorted(set(saved))
if saved:
    print('Saved files:')
    for p in saved:
        print(' -', p)
else:
    print('No season DataFrames found to save. Ensure `dfs` dict or `games_YYYY` globals exist and rerun previous cells.')

Saved files:
 - ../data/processed/structured/games_2016.parquet
 - ../data/processed/structured/games_2017.parquet
 - ../data/processed/structured/games_2018.parquet
 - ../data/processed/structured/games_2019.parquet
 - ../data/processed/structured/games_2020.parquet
 - ../data/processed/structured/games_2021.parquet
 - ../data/processed/structured/games_2022.parquet
 - ../data/processed/structured/games_2023.parquet
 - ../data/processed/structured/games_2024.parquet


In [8]:
games_20['home_team']

0                UAB
1      Southern Miss
2           Marshall
3               Army
4        Texas State
           ...      
537       Penn State
538             UCLA
539       Cincinnati
540          Florida
541     Oregon State
Name: home_team, Length: 542, dtype: object